In [1]:
import pandas as pd
import requests
from tqdm.auto import tqdm
from ipynb.fs.full.db_credenciales import conexion_repo_preparacion

In [ ]:
#import pandas as pd
#import requests
#import os
#import sys
#from tqdm.auto import tqdm
#sys.path.append("C:\\Users\\Yhony\\Documents\\GitHub\\EA_phyton\\Sesiones\\S7")
#from db_credenciales import conexion_repo_preparacion

In [2]:
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

In [3]:
def extract_covid19():

    OWNER = 'CSSEGISandData'
    REPO = 'COVID-19'
    PATH = 'csse_covid_19_data/csse_covid_19_daily_reports'
    URL = f'https://api.github.com/repos/{OWNER}/{REPO}/contents/{PATH}'
    
    response = requests.get(URL)
    con = conexion_repo_preparacion()
    
    print('Conexión exitosa')
    
    i= 0
    for data in tqdm(response.json()):    
        if data['name'].endswith('.csv'):
            file_path = data['download_url']
            dat = extract_from_csv(file_path)
            if i == 0: 
                dat.to_sql(con=con, name='covid19', if_exists='replace')
            else:
                dat.to_sql(con=con, name='covid19', if_exists='append')
            i=i+1
        if i == 10:
            break
        
    print('Extracción y guardado en root exitosa.',)

In [4]:
extract_covid19()

Conexión exitosa


  0%|          | 0/1000 [00:00<?, ?it/s]

Extracción y guardado en root exitosa.
